In [ ]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [ ]:
train_data = MNIST(
    root = "data", 
    train = True,
    transform = ToTensor,
    download = True
)

test_data = MNIST(
    root = "data", 
    train = False,
    transform = ToTensor,
    download = True
)